In [46]:
import leafmap
import streamlit
import geopandas as gpd
import ee
import geemap

In [48]:
# ee.Authenticate()
ee.Initialize()

In [ ]:
farms_gdf = gpd.read_file(r"data\vector\ce_farms.gpkg")
farms = farms_gdf[['farmer', 'crop', 'district', 'province', 'area_ha', 'geometry']]
farms.head()

,farmer,crop,district,province,area_ha,geometry
0,Killarney Farm,Groundnut,Chibombo,Central,20.852368,"MULTIPOLYGON (((28.40682 -14.81873, 28.40445 -..."
1,ADM Farm,Groundnut,Chibombo,Central,53.496625,"MULTIPOLYGON (((28.29156 -14.81992, 28.29139 -..."
2,Simon Hunt,Groundnut,Mpongwe,Copperbelt,19.604325,"MULTIPOLYGON (((28.31578 -13.61503, 28.322 -13..."
3,Isranet Farm,Groundnut,Mpongwe,Copperbelt,49.826289,"MULTIPOLYGON (((28.41546 -13.6795, 28.41705 -1..."
4,Matulufu Farm,Groundnut,Luanshya,Copperbelt,30.613994,"MULTIPOLYGON (((28.49302 -13.18086, 28.49362 -..."


In [63]:
selected_farm_name = "Simon Hunt"

selected_farm = farms[farms["farmer"] == selected_farm_name]
selected_farm_ee = geemap.gdf_to_ee(selected_farm)

selected_start_date = '2024-06-01'
selected_end_date = '2024-07-30'
max_cloud_cover = 20

In [60]:
def get_available_images(selected_farm, selected_start_date, selected_end_date, max_cloud_cover, farms_gdf):
    selected_farm_gdf = farms_gdf[farms_gdf["farmer"] == selected_farm]
    
    proj_selected_farm_gdf = selected_farm_gdf.to_crs(epsg=3857)
    proj_selected_farm_gdf["geometry"] = proj_selected_farm_gdf.geometry.buffer(50)
    buffered_selected_farm_gdf = proj_selected_farm_gdf.to_crs(proj_selected_farm_gdf.crs)

    buffered_selected_farm_ee = geemap.gdf_to_ee(buffered_selected_farm_gdf)

    available_images =  ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterBounds(buffered_selected_farm_ee) \
        .filterDate(selected_start_date, selected_end_date) \
        .select(["B12", "B11", "B8", "B4", "B3", "B2"]) \
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", max_cloud_cover))
    
    available_image = available_images.first().clip(buffered_selected_farm_ee)
    return available_image

In [61]:
available_image = get_available_images(selected_farm_name, selected_start_date, selected_end_date, max_cloud_cover, farms)

In [89]:
min = available_image.select(['B4', 'B3', 'B2']).reduceRegion(
            reducer= ee.Reducer.minMax(),
            geometry= selected_farm_ee,
            scale= 10,
            maxPixels=1e8
        )
min.getInfo()

{'B2_max': 809,
 'B2_min': 503,
 'B3_max': 987,
 'B3_min': 550,
 'B4_max': 1520,
 'B4_min': 513}

In [87]:
n = geemap.Map()
n.add_basemap("ROADMAP")
n.add_ee_layer(available_image, vis_params={"min":200, "max": 2000, "bands": ["B4", "B3", "B2"]}, name ='Image')
n.add_layer(selected_farm_ee, name= 'Farm')
n.zoom_to_gdf(selected_farm)
n

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [44]:
m = leafmap.Map(center= [-13.2, 28], zoom= 6.5, height= "1000px")
m.add_basemap("ROADMAP")
m.add_gdf(selected_farm, zoom_to_layer=True, layer_name="Farms")
# m.to_streamlit(height=700)
m

Map(center=[-13.2, 28], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…